In [1]:
import os
import urllib

import tensorflow as tf
import numpy as np

IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"
if not os.path.exists(IRIS_TRAINING):
    raw = urllib.urlopen(IRIS_TRAINING_URL).read()
    with open(IRIS_TRAINING,'w') as f:
        f.write(raw)

if not os.path.exists(IRIS_TEST):
    raw = urllib.urlopen(IRIS_TEST_URL).read()
    with open(IRIS_TEST,'w') as f:
        f.write(raw)

In [3]:
BATCH_SIZE = 10
N_FEATURES = 4
def batch_generator(filenames,batchsize):
    filename_queue = tf.train.string_input_producer(filenames)
    reader = tf.TextLineReader(skip_header_lines=1) # skip the first line in the file,queue
    _, value = reader.read(filename_queue)

    record_defaults = [[1.0],[1.0],[1.0],[1.0],[1]]
    
    # read in the 10 rows of data
    content = tf.decode_csv(value, record_defaults=record_defaults) 
    features = tf.stack(content[:N_FEATURES])
    # assign the last column to label
    label = content[-1]
    data_batch, label_batch = tf.train.batch([features, label], batch_size=BATCH_SIZE)#queue
    return data_batch, label_batch

In [4]:
data_train, label_train = batch_generator(['iris_training.csv'])#这些都是Op，可以直接链接到需要数据的地方
data_test, label_test = batch_generator(['iris_test.csv'])

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    for _ in range(15): # generate 10 batches
        features, labels = sess.run([data_train, label_train])
        print(features)
    coord.request_stop()
    coord.join(threads)

TypeError: batch_generator() missing 1 required positional argument: 'batchsize'

这里要提到一点：可以用read_up_to可以加速一个reader来源的数据读入速度，比read+train.batch快很多